In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# 필요한 모듈 임포트  
import pandas as pd  
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPooling1D, concatenate


In [2]:
# 데이터 읽어오기
train_file = "data/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()



    Pandas의 read_csv()함수를 이용해 chatbot_data.csv파일을 읽어와 
    CNN 모델 학습시 필요한 Q(질문)와 label(감정) 데이터를 features와 labels 리스트에 저장합니다.


In [4]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
# 문장 내 모든 단어를 시퀀스 번호로 변환
sequences = tokenizer.texts_to_sequences(corpus)

In [ ]:
corpus
sequences

    질문 리스트(features)에서 문장을 하나씩 꺼내와 text_to_word_sequence() 함수를 이용해 단어 시퀀스를 만든다.
    이렇게 생성된 단어 시퀀스를 말뭉치(corpus) 리스트에 저장합니다. 
    그 다음 텐서플로 토크나이저의 text_to_sequences() 함수를 이용해 문장 내 모든 단어를 시퀀스 번호로 변환 
    우리는 변환된 시퀀스 번호를 이용해 단어 임베딩 벡터를 만들 것입니다.


In [5]:
word_index = tokenizer.word_index


In [6]:
# 너무 크지 않게 잡아야 한다 15-20 사이로 지정
MAX_SEQ_LEN = 15  # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences,
                                                   maxlen=MAX_SEQ_LEN,
                                                   padding='post')

    시퀀스 번호로 만든 벡터는 한 가지 문제가 있습니다. 바로 문장의 길이가 제각각이기 때문에 벡터 크기가 다 다릅니다. 
    CNN모델의 입력 계층은 고정된 개수의 입력 노드를 가지고 있습니다.
    시퀀스 번호로 변환된 전체 벡터 크기를 동일하게 맞춰야 합니다. 
    예제에서는 MAX_SEQ_LEN 크기만큼 넉넉하게 늘립니다. 

    이 경우  MAX_SEQ_LEN 크기보다 작은 벡터에는 남는 공간이 생기는데, 
    이를 0으로 채우는 작업을 해야 합니다. 이런 일련의 과정을 패딩(𝑝𝑎𝑑𝑑𝑖𝑛𝑔)처리라고 합니다. 

    케라스에서는 pad_sequences() 함수를 통해 시퀀스의 패딩 처리를 손쉽게 할 수 있습니다. 

    pad_sequences() 함수를 사용할 때 maxlen 인자  로 시퀀스의 최대 길이를 정하는데, 
    학습시킬 문장 데이터들을 사전에 분석해 최대 몇 개의 단어 토큰으로 구성되어 있는지 파악해야 합니다. 

    너무 크게 잡으면 빈 공간이 많이 생겨 자원의 낭비가 발생합니다. 
    반대로 너무 작게 잡으면 입력데이터가 손상되는 상황이 발생하게 됩니다.

In [9]:
# 학습용, 검증용, 테스트용 데이터셋 생성 # 학습셋:검증셋:테스트셋 = 7:2:1  
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))  
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs) * 0.7)  
val_size = int(len(padded_seqs) * 0.2)  
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)


    앞에서 패딩 처리된 시퀀스(padded_seqs) 벡터 리스트와 감정(labels), 리스트 전체를 데이터셋 객체로 만듭니다. 
    그 다음 데이터를 랜덤으로 섞은 후 학습용, 검증용, 테스트용 데이터셋을 7:2:1비율로 나눠 실제 학습에 필요한 데이터셋 객체를 각각 분리합니다.


In [10]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1  # 전체 단어 수



In [11]:
# CNN 모델 정의

# shape 인자로 입력 노드에 들어올 데이터의 형상( 𝑠ℎ𝑎𝑝𝑒 )을 지정
# 실제 패딩 처리된 시퀀스 벡터의 크기(MAX_SEQ_LEN)로 설정합니다.
input_layer = Input(shape=(MAX_SEQ_LEN, ))

# 임베딩 계층은 희소 벡터를 입력받아 데이터 손실을 최소화하면서 벡터 차원이 압축되는 밀집 벡터로 변환
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE,
                            input_length=MAX_SEQ_LEN)(input_layer)

# 단어 임베딩 부분의 마지막에는 50% 확률로 Dropout()을 생성
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

# 임베딩 계층을 통해 전달된 임베딩 벡터에서 특징 추출을 하는 영역을 구현
# 합성곱 연산 과정을 떠올려보면 필터 크기에 맞게 입력 데이터 위를 슬라이딩하게 되는데, 
# 이는 3,4,5-gram  언어 모델의 개념과 비슷
conv1 = Conv1D(filters=128,
               kernel_size=3,
               padding='valid',
               activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPooling1D()(conv1)

conv2 = Conv1D(filters=128,
               kernel_size=4,
               padding='valid',
               activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPooling1D()(conv2)

conv3 = Conv1D(filters=128,
               kernel_size=5,
               padding='valid',
               activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPooling1D()(conv3)

# 3, 4, 5- gram 이후 합치기
# 각각 병렬로 처리된 합성곱 계층의 특징맵 결과를 하나로 묶어줍니다.
concat = concatenate([pool1, pool2, pool3])  

# 완전 연결 계층을 구현
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)  

# 챗봇 데이터 문장에서 3가지 클래스로 감정 분류해야 하기 때문에 출력 노드가 3개인 Dense()를 생성
logits = Dense(3, name='logits')(dropout_hidden)

# logits에서 나온 점수로 소프트맥스 계층을 통해 감정 클래스별 확률을 계산
predictions = Dense(3, activation=tf.nn.softmax)(logits)


    문장을 감정 클래스로 분류하는 CNN모델은 전처리된 입력 데이터를 단어 임베딩 처리하는 영역과 
    합성곱 필터와 연산을 통해 문장의 특징 정보(특징맵)을 추출하고, 평탄화(𝑓𝑙𝑎𝑡𝑡𝑒𝑛)를 하는 영역, 
    그리고 완전 연결 계층(𝑓𝑢𝑙𝑙𝑦 𝑐𝑜𝑛𝑛𝑒𝑐𝑡𝑒𝑑 𝑙𝑎𝑦𝑒𝑟)을 통해 감정별로 클래스를 분류하는 영역으로 구성되어 있습니다. 
    
    클래스 분류 모델(𝑐𝑙𝑎𝑠𝑠𝑖𝑓𝑖𝑐𝑎𝑡𝑖𝑜𝑛 𝑝𝑟𝑜𝑏𝑙𝑒𝑚)을 학습할 때 주로 손실값(𝑙𝑜𝑠𝑠)을 계산하는 함수로 sparse_categorical_crossentropy를 사용합니다. 이때 크로스 엔트로피(𝑐𝑟𝑜𝑠𝑠 𝑒𝑛𝑡𝑟𝑜𝑝𝑦)을 위해 확률값을 입력으로 사용해야 하는데 이를 위해 소프트맥스 계층이 필요합니다!

In [12]:
# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    앞에서 정의한 계층들을 케라스 모델에 추가하는 작업을 해야합니다. 
    케라스 모델을 생성할 때 Model()을 사용하는데, 인자로는 앞서 생성한 입력 계층(input_layer)과 출력 계층(predictions)을 사용
    모델 정의 후 실제 모델을 model.compile() 함수를 통해 CNN 모델을 컴파일 합니다.  
    최적화(𝑜𝑝𝑡𝑖𝑚𝑖𝑧𝑒𝑟) 방법에는 𝑎𝑑𝑎𝑚을, 손실 함수에는 sparse_categorical_crossentropy를 사용하도록 했습니다. 
    또한 케라스 모델을 평가할 때 정확도를 확인하기 위해 metrics에 accuracy를 사용했습니다.


In [13]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
414/414 [==============================] - 41s 20ms/step - loss: 0.9827 - accuracy: 0.4973 - val_loss: 0.6599 - val_accuracy: 0.6557
Epoch 2/5
414/414 [==============================] - 8s 19ms/step - loss: 0.5846 - accuracy: 0.7549 - val_loss: 0.3036 - val_accuracy: 0.9074
Epoch 3/5
414/414 [==============================] - 8s 19ms/step - loss: 0.3280 - accuracy: 0.8868 - val_loss: 0.1721 - val_accuracy: 0.9501
Epoch 4/5
414/414 [==============================] - 8s 19ms/step - loss: 0.1880 - accuracy: 0.9430 - val_loss: 0.0954 - val_accuracy: 0.9729
Epoch 5/5
414/414 [==============================] - 8s 19ms/step - loss: 0.1331 - accuracy: 0.9587 - val_loss: 0.0628 - val_accuracy: 0.9810


    정의한 CNN 모델을 학습합니다. 
    첫 번째 인자에는 학습용 데이터셋을 입력하고, 두 번째 validation_data 인자에는 검증용 데이터셋을 입력합니다. 
    예제에서는 에포크값을 5로 설정했으므로 모델 학습을 5회 반복합니다. 
    verbose 인자가 1인 경우에는 모델 학습 시 진행 과정을 상세하게 보여줍니다.  
    만약 생략을 원한다면 0으로 맞춰주시면 됩니다. 
    그리고 학습 model.fit() 함수 호출 후 마지막 에포크 단계일 때 보이는 결과 화면 입니다.

In [14]:
# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)  
print('Accuracy: %f' % (accuracy * 100))  
print('loss: %f' % (loss))


60/60 [==============================] - 0s 3ms/step - loss: 0.0456 - accuracy: 0.9865
Accuracy: 98.646361
loss: 0.045564


    evaluate() 함수를 이용해 성능을 평가합니다. 
    evaluate() 함수의 인자로 테스트용 데이터셋을 사용합니다.


In [15]:
# 모델 저장
model.save('cnn_model.h5')

    학습이 완료된 모델을 h5 파일 포맷으로 저장합니다. 
    이후 저장된 모델 파일을 불러와 문장 데이터의 감정 분류를 할 예정입니다.


In [ ]:
# 필요한 모듈 임포트  
import pandas as pd  
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPooling1D, concatenate


# 데이터 읽어오기
train_file = "data/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]

# 문장으로부터 단어를 토큰화하고 숫자에 대응시키는 딕셔너리를 사용
tokenizer = preprocessing.text.Tokenizer()
# 문자 데이터를 입력받아서 리스트의 형태로 변환
tokenizer.fit_on_texts(corpus)
# 문장 내 모든 단어를 시퀀스 번호로 변환
sequences = tokenizer.texts_to_sequences(corpus)
# 단어와 숫자의 키-값 쌍을 포함하는 딕셔너리를 반환
word_index = tokenizer.word_index


# 패딩 처리 / 너무 크지 않게 잡아야 한다 15-20 사이로 지정
MAX_SEQ_LEN = 15  # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences,
                                                   maxlen=MAX_SEQ_LEN,
                                                   padding='post')



# 데이터셋 나누기 위한 전처리
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))  
ds = ds.shuffle(len(features))

# 학습용, 검증용, 테스트용 7:2:1
train_size = int(len(padded_seqs) * 0.7)  
val_size = int(len(padded_seqs) * 0.2)  
test_size = int(len(padded_seqs) * 0.1)

# 학습용, 검증용, 테스트용 데이터셋 나누기
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)



# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1  # 전체 단어 수


# CNN 모델 정의

# 입력 노드 : 입력 노드에 들어올 데이터의 형상( 𝑠ℎ𝑎𝑝𝑒 )을 실제 패딩 처리된 시퀀스 벡터의 크기(MAX_SEQ_LEN)로 설정
input_layer = Input(shape=(MAX_SEQ_LEN, ))

# 임베딩 계층은 희소 벡터를 입력받아 데이터 손실을 최소화하면서 벡터 차원이 압축되는 밀집 벡터로 변환
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE,
                            input_length=MAX_SEQ_LEN)(input_layer)

# 단어 임베딩 부분의 마지막에는 50% 확률로 Dropout()을 생성
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

# 임베딩 계층을 통해 전달된 임베딩 벡터에서 특징 추출을 하는 영역을 구현
# 이는 3,4,5-gram  언어 모델의 개념과 비슷
conv1 = Conv1D(filters=128,
               kernel_size=3,
               padding='valid',
               activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPooling1D()(conv1)

conv2 = Conv1D(filters=128,
               kernel_size=4,
               padding='valid',
               activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPooling1D()(conv2)

conv3 = Conv1D(filters=128,
               kernel_size=5,
               padding='valid',
               activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPooling1D()(conv3)


# 각각 병렬로 처리된 합성곱 계층의 특징맵 결과를 하나로 합친다.
concat = concatenate([pool1, pool2, pool3])  

# 완전 연결 계층을 구현
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)  

# 출력 노드가 3개인 Dense()를 생성
logits = Dense(3, name='logits')(dropout_hidden)

# logits에서 나온 점수로 소프트맥스 계층을 통해 감정 클래스별 확률을 계산
predictions = Dense(3, activation=tf.nn.softmax)(logits)

# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)  
print('Accuracy: %f' % (accuracy * 100))  
print('loss: %f' % (loss))


# 모델 저장
model.save('cnn_model.h5')


#### 감정 분류

In [3]:
import tensorflow as tf  
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

# 데이터 읽어오기
train_file = "data/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()  
labels = data['label'].tolist()



        pandas의 read_csv()함수를 이용해 chatbot_data.csv 파일을 읽어와 
        label(감정)을 분류할 Q(질문) 데이터를 features 리스트에 저장합니다.
        labels 리스트는 CNN 모델이 예측한 분류 결과와 실제 분류값을 비교하기 위한 결과로 사용됩니다.


In [4]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()  
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
# print(sequences)


MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')
# print(padded_seqs)

        앞서 불러온 질문 리스트(features)에서 한 문장씩 꺼내와 
        text_to_word_sequence() 함수를 이용해 단어 시퀀스를 만든 후 말뭉치(corpus) 리스트에 저장합니다.

        그 다음으로 텐서플로 토크나이저의 texts_to_sequences()함수를 이용해 문장 내 모든 단어를 시퀀스 번호로 변환합니다.

        마지막으로 단어 시퀀스 벡터 크기를 맞추기 위해 pad_sequences()함수를 사용하여 패딩 처리합니다.


In [18]:
# 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))  
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20) # 테스트 데이터셋

        앞에서 패딩 처리한 시퀀스(padded_seqs) 벡터 리스트와 감정(labels) 리스트 전체를 데이터셋 객체로 만듭니다. 
        그 다음에는 데이터를 랜덤으로 섞은 후 테스트용 데이터셋 2,000개 뽑아내 20개씩 배치 처리합나다.


In [19]:
# 감정 분류 CNN 모델 불러오기
model = load_model('data/cnn_model.h5')  
model.summary()
model.evaluate(test_ds, verbose=2)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 15, 128)      1715072     input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 15, 128)      0           embedding[0][0]                  
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 13, 128)      49280       dropout[0][0]                    
______________________________________________________________________________________________

[0.06204930320382118, 0.9785000085830688]

        케라스의 load_model() 함수를 이용해 모델 파일을 불러옵니다.
        성공적으로 모델 파일을 불러왔다면 학습된 모델 객체를 반환합니다. 
        파일에 저장된 모델  정보를 확인하기 위해 summary() 함수를 호출하고, 
        테스트셋 데이터를 이용해 모델 성능을 평가합니다.


In [20]:
# 테스트용 데이터셋의 10212번째 데이터 출력  
print("단어 시퀀스 : ", corpus[10212])  
print("단어 인덱스 시퀀스 : ", padded_seqs[10212])  
print("문장 분류(정답) : ", labels[10212])


단어 시퀀스 :  ['썸', '타는', '여자가', '남사친', '만나러', '간다는데', '뭐라', '해']
단어 인덱스 시퀀스 :  [   13    61   127  4320  1333 12162   856    31     0     0     0     0
     0     0     0]
문장 분류(정답) :  2


In [21]:
# 테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks])  
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())

감정 예측 점수 :  [[3.4567800e-06 1.0106982e-06 9.9999559e-01]]
감정 예측 클래스 :  [2]
